### Imports and settings

In [ ]:
# file system navigation
from pathlib import Path

# data transformation
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
%matplotlib inline

# ml algorithms and evaluation metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn import model_selection
from sklearn.model_selection import StratifiedShuffleSplit
import scipy
from scipy.stats.distributions import uniform, randint

# sklearn specifics
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.exceptions import DataConversionWarning
from sklearn import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# nlp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from spacy.pipeline import TextCategorizer
from spacy.util import minibatch, compounding
from spacy.util import decaying

# keras
from keras import datasets
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

# misc
import random
import copy
import pickle
import re
from collections import namedtuple
from typing import List

In [ ]:
from helpers import *

In [ ]:
import tf_helpers

In [ ]:
DATA_PATH = Path.cwd() / "data" / "shared"
OUTPUT_PATH = Path.cwd() / "reports" / "images-and-tables"
MODEL_PATH = Path.cwd() / "models"
SEED = 5

In [ ]:
TEXT_COL = "text"
NUM_COLS = ["claps", "reading_time"]

In [ ]:
if not OUTPUT_PATH.is_dir():
    OUTPUT_PATH.mkdir(parents=True)
    
if not MODEL_PATH.is_dir():
    MODEL_PATH.mkdir(parents=True)

### Helper visualizations

#### Interest in ML

Generate a plot of interest in Machine Learning over time based on Google trends data

In [ ]:
google_trends_ml = pd.read_csv(Path.cwd() / "data" / "trends-ml.csv",
                               skiprows=3,
                               header=None,
                               names=["date", "interest"],
                               parse_dates=["date"])

In [ ]:
google_trends_ml.head()

In [ ]:
interest_plot = google_trends_ml.plot(x="date",
                      y="interest",
                      legend=False)
interest_plot.set_xlabel("Date")
interest_plot.set_ylabel("Relative interest")
interest_plot;

In [ ]:
fig = interest_plot.get_figure()
fig.savefig(OUTPUT_PATH / "interest-in-ml.png")

### Data preparation

#### Load preprocessed data and split into train and test

In [ ]:
data = pd.read_parquet(DATA_PATH / "train_data.parquet")

In [ ]:
X = data[["claps", "reading_time", "text"]]
y = np.array(data["interesting"])

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3, random_state=SEED,
                                                    stratify=y)

Initialize custom evaluator and shuffle split generator to use across all modeling approaches below.

In [ ]:
evaluator = CustomEvaluator(target_precision=0.8)
sss = model_selection.StratifiedShuffleSplit(n_splits=6, test_size=0.2, random_state=SEED)

#### Generate a vocabulary of words specific to the corpus

We use a list of the top 10k most frequent words in the English language obtained from [this repository](https://github.com/first20hours/google-10000-english) to identify the words specific to the corpus of block posts.

In [ ]:
TOP_WORDS_PATH = Path.cwd() / "resources" / "top_words"
top_10k = pd.read_table(TOP_WORDS_PATH / "google-10000-english" / "google-10000-english.txt", header=None)
top_10k_dict = {str(word).lower() : rank + 1 for rank, word in top_10k.iloc[:, 0].items()}

In [ ]:
texts = data["text"].copy()

In [ ]:
texts = texts.apply(lambda x: x.lower())
texts = texts.apply(lambda x: clean_apostrophe(x))
texts = texts.apply(lambda x: remove_punctuation(x))
texts = texts.apply(lambda x: fix_specific(x))

In [ ]:
sentences = texts.apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
oov = check_coverage(vocab, top_10k_dict)

In [ ]:
specific_vocab = [w for w, _ in oov]

### Data exploration

#### Summary statistics

Take a first look at the data and generate summary tables for the report

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data["interesting"].value_counts() / data.shape[0]

In [ ]:
summary_numeric = (data[["claps", "reading_time"]]
                   .describe()
                   .round(2))
summary_numeric

In [ ]:
save_html(df=summary_numeric, name="summary_numeric", out=OUTPUT_PATH)

In [ ]:
summary_object = data[["author", "title", "text"]].describe()
summary_object

In [ ]:
save_html(df=summary_object, name="summary_object", out=OUTPUT_PATH)

#### Distribution of target with respect to numerical features

In [ ]:
data_base = data[["claps", "reading_time", "interesting"]]

In [ ]:
x_index = 0
y_index = 1
target_names = ["not interesting", "interesting"]

colors = ["red", "green"]

for label, color in zip(range(len(data_base["interesting"])), colors):
    plt.scatter(np.array(data_base[data_base["interesting"]==label].iloc[:, x_index]), 
                np.array(data_base[data_base["interesting"]==label].iloc[:, y_index]),
                label=target_names[label],
                c=color)

plt.xlabel(data_base.columns[x_index])
plt.ylabel(data_base.columns[y_index])
plt.legend(loc="upper right")

plt.savefig(OUTPUT_PATH / "base_classifier.png")

#### Distribution of blog post lengths

In [ ]:
n_words = [len(post.split(" ")) for post in data["text"]]
words_per_review = len(data) / np.median(n_words)
plt.hist(n_words, 30)
plt.xlabel("Number of words")
plt.ylabel("Number of blog posts")
plt.xlim(0, 8000)
plt.title(f"Blog post number of words distribution (ratio: {words_per_review:.2f})")
plt.savefig(OUTPUT_PATH / "blog-post-length.png")

In [ ]:
print("Summary statistics on the number of words:")
print(f"   Median: {np.median(n_words):.2f}")
print(f"   Mean: {np.mean(n_words):.2f}")

In [ ]:
print(f"Number of samples / median number of words in each post: {words_per_review:.2f}")

#### For comparison: Distribution length for imdb data

In [ ]:
imdb = datasets.imdb
(train_imdb, _), (test_imdb, _) = imdb.load_data(Path.cwd() / "data" / "imdb.npz", num_words=None)

In [ ]:
imdb_full = np.concatenate((train_imdb, test_imdb))

In [ ]:
n_words_imdb = [len(review) for review in imdb_full]
words_per_review_imdb = imdb_full.shape[0] / np.median(n_words_imdb)
plt.hist(n_words, 50)
plt.xlabel("Number of words")
plt.ylabel("Number of review")
plt.xlim(0, 8000)
plt.title(f"Imdb review number of words distribution (ratio: {words_per_review_imdb:.2f})")
plt.savefig(OUTPUT_PATH / "imdb-review-length.png")

In [ ]:
print("Summary statistics on the number of words for imdb:")
print(f"   Median: {np.median(n_words_imdb):.2f}")
print(f"   Mean: {np.mean(n_words_imdb):.2f}")

In [ ]:
print(f"Number of samples / median number of words in each review: {words_per_review_imdb:.2f}")

### Baseline model

Create a baseline model using just the numerical features `claps` and `reading time` based on three classes of classification models:

- Random forests
- Support vector machines
- Logistic regression

In [ ]:
X_train_num, X_test_num = np.array(X_train[num_cols]), np.array(X_test[num_cols])

#### Random Forest 

In [ ]:
rf = RandomForestClassifier(n_estimators=20,
                            min_samples_leaf=3,
                            random_state=SEED)

In [ ]:
fitted_rfs = fit_ensemble(rf, sss, X_train_num, y_train)

In [ ]:
evaluate_ensemble(fitted_rfs, evaluator, X_test_num, y_test)

In [ ]:
base_results_rf = evaluate_ensemble(fitted_rfs, evaluator, X_test_num, y_test, return_res=True, method="baseline_rf")

#### Support Vector Machine

In [ ]:
svc = SVC(gamma="auto",
          probability=True)

In [ ]:
fitted_svcs = fit_ensemble(svc, sss, X_train_num, y_train)

In [ ]:
evaluate_ensemble(fitted_svcs, evaluator, X_test_num, y_test)

In [ ]:
base_results_svc = evaluate_ensemble(fitted_svcs, evaluator, X_test_num, y_test, return_res=True, method="baseline_svc")

#### Logistic regression

In [ ]:
lr = LogisticRegression(solver="liblinear", random_state=SEED)

In [ ]:
fitted_lrs = fit_ensemble(lr, sss, X_train_num, y_train)

In [ ]:
evaluate_ensemble(fitted_lrs, evaluator, X_test_num, y_test)

In [ ]:
base_results_lr = evaluate_ensemble(fitted_lrs, evaluator, X_test_num, y_test, return_res=True, method="baseline_lr")

#### Collect and save baseline results

In [ ]:
base_results = pd.concat([base_results_rf, base_results_svc, base_results_lr], axis=0)

In [ ]:
base_results

In [ ]:
save_html(df=base_results, name="summary_baseline_results", out=OUTPUT_PATH, index=False)

### Text based models

Prepare feature array for training text based models by extracting just the column containing the blog posts' text from `X_train`.

In [ ]:
X_train_text, X_test_text = np.array(X_train[TEXT_COL]), np.array(X_test[TEXT_COL])

#### CountVectorizer + Classifier

Scikit-learn's `CountVectorizer` is the simplest approach to turning the blog posts' texts into numerical matrices. It will just count the number of occurences of each token in the text and create a sparse matrix holding these counts for all posts.

##### Default values

First, let's do everything with default values to get a general feeling for how this approach performs.

In [ ]:
count_vectorizer_specific = CountVectorizer(vocabulary=specific_vocab)

In [ ]:
count_vectorizer = CountVectorizer()

###### Random Forest

With full vocab

In [ ]:
pipe_countvec_rf = make_pipeline(count_vectorizer, RandomForestClassifier(n_estimators=10,
                                                                          random_state=SEED,
                                                                          n_jobs=-1))
fitted_countvec_rf = fit_ensemble(pipe_countvec_rf, sss, X_train_text, y_train)
res_countvec_rf_full = evaluate_ensemble(fitted_countvec_rf, evaluator, X_test_text, y_test,
                                         return_res=True, method=f"countvec rf full v{SEED}")

In [ ]:
save_pickle(res_countvec_rf_full, OUTPUT_PATH)

Only top k words specific to the corpus

In [ ]:
pipe_countvec_rf_specific = make_pipeline(count_vectorizer_specific, RandomForestClassifier(n_estimators=10,
                                                                                            random_state=1,
                                                                                            n_jobs=-1))
fitted_countvec_rf_specific = fit_ensemble(pipe_countvec_rf_specific, sss, X_train_text, y_train)
res_countvec_rf_specific = evaluate_ensemble(fitted_countvec_rf_specific, evaluator, X_test_text, y_test,
                                             return_res=True, method=f"countvec rf specific v{SEED}")

In [ ]:
save_pickle(res_countvec_rf_specific, OUTPUT_PATH)

###### Support Vector Machine

We use scikit-learn's `StandardScaler` here as the SVC's default kernel (`rbf`) expects normalized features.

In [ ]:
pipe_countvec_svc = make_pipeline(count_vectorizer, StandardScaler(with_mean=False),
                                  SVC(gamma="auto", probability=True, random_state=SEED))
fitted_countvec_svc = fit_ensemble(pipe_countvec_svc, sss, X_train_text, y_train)
res_countvec_svc_full = evaluate_ensemble(fitted_countvec_svc, evaluator, X_test_text, y_test,
                                          return_res=True, method=f"countvec svc full v{SEED}")

In [ ]:
save_pickle(res_countvec_svc_full, OUTPUT_PATH)

In [ ]:
pipe_countvec_svc_specific = make_pipeline(count_vectorizer_specific,
                                           StandardScaler(with_mean=False),
                                           SVC(gamma="auto", probability=True, random_state=SEED))
fitted_countvec_svc_specific = fit_ensemble(pipe_countvec_svc_specific, sss, X_train_text, y_train)
res_countvec_svc_specific = evaluate_ensemble(fitted_countvec_svc_specific, evaluator, X_test_text, y_test,
                                              return_res=True, method=f"countvec svc specific v{SEED}")

In [ ]:
save_pickle(res_countvec_svc_specific, OUTPUT_PATH)

##### Grid search on best default models

###### Full vocab + SVC

In [ ]:
pipe = Pipeline([
    ("vec", CountVectorizer()),
    ("std", StandardScaler(with_mean=False)),
    ("svc", SVC(probability=True))
    ])
params = {"vec__stop_words": ["english", None],
          "vec__ngram_range": [(1, 1), (1, 2), (1, 3)], 
          "vec__max_df": uniform(loc=0.8, scale=0.2),
          "vec__min_df": uniform(loc=0.0, scale=0.2),
          "vec__max_features": randint(low=1000, high=20000),
          "svc__C": scipy.stats.expon(scale=1.0),
          "svc__gamma": ["auto", "scale"],
          "svc__kernel": ["rbf"],
          "svc__class_weight": ["balanced", None]}

grid = RandomizedSearchCV(pipe,
                          params,
                          n_iter=50,
                          scoring="roc_auc",
                          cv=5,
                          return_train_score=False)

In [ ]:
grid_fitted = grid.fit(X_train_text, y_train)

In [ ]:
joblib.dump(grid_fitted.best_estimator_, MODEL_PATH / "countvec_full_svc_grid_best.pkl", compress=1)

In [ ]:
countvec_svc_full_grid_best = joblib.load(MODEL_PATH / "countvec_full_svc_grid_best.pkl")
countvec_svc_full_grid_best = countvec_svc_full_grid_best.set_params(svc__random_state=SEED)
fitted_countvec_svc_full_grid_best = fit_ensemble(countvec_svc_full_grid_best, sss,
                                                X_train_text, y_train, print_progress=True)
res_countvec_svc_full_best = evaluate_ensemble(fitted_countvec_svc_full_grid_best, evaluator, X_test_text, y_test,
                                              return_res=True, method=f"countvec svc full best params v{SEED}") 

In [ ]:
countvec_svc_full_grid_best.get_params()

In [ ]:
save_pickle(res_countvec_svc_full_best, OUTPUT_PATH)

###### Specific vocab + SVC

In [ ]:
pipe = Pipeline([
    ("vec", CountVectorizer(vocabulary=specific_vocab)),
    ("std", StandardScaler(with_mean=False)),
    ("svc", SVC(probability=True))
    ])
params = {"vec__stop_words": ["english", None],
          "vec__ngram_range": [(1, 1), (1, 2), (1, 3)], 
          "vec__max_df": uniform(loc=0.8, scale=0.2),
          "vec__min_df": uniform(loc=0.0, scale=0.2),
          "vec__max_features": randint(low=1000, high=20000),
          "svc__C": scipy.stats.expon(scale=1.0),
          "svc__gamma": ["auto", "scale"],
          "svc__kernel": ["rbf"],
          "svc__class_weight": ["balanced", None]}

grid = RandomizedSearchCV(pipe,
                          params,
                          n_iter=50,
                          scoring="roc_auc",
                          cv=5,
                          return_train_score=False)

In [ ]:
grid_fitted = grid.fit(X_train_text, y_train)

In [ ]:
joblib.dump(grid_fitted.best_estimator_, MODEL_PATH / "countvec_specific_svc_grid_best.pkl", compress=1)

In [ ]:
countvec_svc_specific_grid_best = joblib.load(MODEL_PATH / "countvec_specific_svc_grid_best.pkl")
countvec_svc_specific_grid_best = countvec_svc_specific_grid_best.set_params(svc__random_state=SEED)
fitted_countvec_svc_specific_grid_best = fit_ensemble(countvec_svc_specific_grid_best, sss,
                                                X_train_text, y_train, print_progress=True)
res_countvec_svc_specific_best = evaluate_ensemble(fitted_countvec_svc_specific_grid_best, evaluator, X_test_text, y_test,
                                              return_res=True, method=f"best params countvec svc specific v{SEED}") 

In [ ]:
countvec_svc_specific_grid_best.get_params()

In [ ]:
save_pickle(res_countvec_svc_specific_best, OUTPUT_PATH)

###### Create summary of grid search results and save to html

In [ ]:
list_countvec_grid = read_results(OUTPUT_PATH / "raw", "best_params_countvec*.pkl")

In [ ]:
res = pd.concat(list_countvec_grid)

In [ ]:
res_groupby_method = res.groupby("base_method")
res_analysis_table = res_groupby_method.agg({c : ["mean", "std"] for c in ["mean train auc", "mean cv auc", "mean test auc"]})

In [ ]:
out = res_analysis_table.sort_values([("mean test auc", "mean"), ("mean cv auc", "mean")], axis=0, ascending=False)
out.index.name = None

In [ ]:
index = pd.MultiIndex.from_product([["train auc", "cv auc", "test auc"], ["mean", "std"]])
out.columns = index

In [ ]:
out

In [ ]:
save_html(out, "summary_countvec_svc_best_params", out=OUTPUT_PATH, index=True)

#### TfidfVectorizer + Classifier

The next approach to feature extraction from the text data that I want to try is the `Term-Frequency-Inverse-Document-Frequency` technique implemented in scikit-learn's `TfidfVectorizer`. This method creates the same matrix as the `CountVectorizer` but divides the values for each token in the vocabulary by its frequency across all documents in the corpus.

##### Default values

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer_specific = TfidfVectorizer(vocabulary=specific_vocab)

###### Random Forest

Full vocab

In [ ]:
pipe_tfidf = make_pipeline(tfidf_vectorizer, RandomForestClassifier(n_estimators=10,
                                                                    random_state=SEED,
                                                                    n_jobs=-1))
fitted_tfidf = fit_ensemble(pipe_tfidf, sss, X_train_text, y_train)
res_tfidf_rf_full = evaluate_ensemble(fitted_tfidf, evaluator, X_test_text, y_test,
                                      return_res=True, method=f"tfidf rf full v{SEED}")

In [ ]:
save_pickle(res_tfidf_rf_full, OUTPUT_PATH)

Only specific vocab

In [ ]:
pipe_tfidf_specific = make_pipeline(tfidf_vectorizer_specific, RandomForestClassifier(n_estimators=10,
                                                                                      random_state=SEED,
                                                                                      n_jobs=-1))
fitted_tfidf_specific = fit_ensemble(pipe_tfidf_specific, sss, X_train_text, y_train)
res_tfidf_rf_specific = evaluate_ensemble(fitted_tfidf_specific, evaluator, X_test_text, y_test,
                                          return_res=True, method=f"tfidf rf specific v{SEED}")

In [ ]:
save_pickle(res_tfidf_rf_specific, OUTPUT_PATH)

###### Support Vector Machine

In [ ]:
pipe_tfidf_svc = make_pipeline(tfidf_vectorizer,
                               StandardScaler(with_mean=False),
                               SVC(gamma="auto",
                                   probability=True,
                                   random_state=SEED))
fitted_tfidf_svc = fit_ensemble(pipe_tfidf_svc, sss, X_train_text, y_train)
res_tfidf_svc_full = evaluate_ensemble(fitted_tfidf_svc, evaluator, X_test_text, y_test,
                                       return_res=True, method=f"tfidf svc full v{SEED}")

In [ ]:
save_pickle(res_tfidf_svc_full, OUTPUT_PATH)

In [ ]:
pipe_tfidf_svc_specific = make_pipeline(tfidf_vectorizer_specific,
                                        StandardScaler(with_mean=False),
                                        SVC(gamma="auto",
                                            probability=True,
                                            random_state=SEED))
fitted_tfidf_svc_specific = fit_ensemble(pipe_tfidf_svc_specific, sss, X_train_text, y_train)
res_tfidf_svc_specific = evaluate_ensemble(fitted_tfidf_svc_specific, evaluator, X_test_text, y_test,
                                           return_res=True, method=f"tfidf svc specific v{SEED}")

In [ ]:
save_pickle(res_tfidf_svc_specific, OUTPUT_PATH)

##### Collect results from running the above with different seeds

In [ ]:
list_countvec = read_results(OUTPUT_PATH / "raw", "countvec*.pkl")

In [ ]:
list_tfidf = read_results(OUTPUT_PATH / "raw", "tfidf*.pkl")

In [ ]:
results_tfidf_countvec = pd.concat([e for l in [list_countvec, list_tfidf] for e in l])

In [ ]:
results_tfidf_countvec

In [ ]:
save_html(df=results_tfidf_countvec.drop("base_method", axis=1).sort_values(["mean test auc", "mean cv auc"], axis=0, ascending=False),
          name="countvec_tfidf_results_default_all_appendix",
          out=OUTPUT_PATH,
          index=False)

##### Summarize results and save overview table

In [ ]:
res = results_tfidf_countvec.sort_values(["mean test auc", "mean cv auc"], axis=0, ascending=False)

In [ ]:
res_groupby_method = res.groupby("base_method")
res_analysis_table = res_groupby_method.agg({c : ["mean", "std"] for c in ["mean train auc", "mean cv auc", "mean test auc"]})

In [ ]:
out = res_analysis_table.sort_values([("mean test auc", "mean"), ("mean cv auc", "mean")], axis=0, ascending=False)
out.index.name = None

In [ ]:
index = pd.MultiIndex.from_product([["train auc", "cv auc", "test auc"], ["mean", "std"]])
out.columns = index

In [ ]:
out

In [ ]:
save_html(df=out,
          name="summary_countvec_tfidf_results_default",
          out=OUTPUT_PATH,
          index=True)

#### Pretrained word embeddings + neural network 

The below is heavily based on these two Kaggle Kernels:
- [Processing text when using word embeddings](https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings/notebook)
- [Comparing word embeddings](https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings/notebook)

##### Config

In [ ]:
text_len = data["text"].apply(lambda x: len(x.split(" ")))
text_para_len = data["text"].apply(lambda x: len(x.split("\n")))

In [ ]:
print(f"There are {len(vocab)} distinct words in the blog posts.")
print()
print("Summary statistics on the number of words across all blog posts:")
print(f"   Mean: {text_len.mean():.2f}")
print(f"   Median: {text_len.median():.2f}")
print(f"   Maximum: {text_len.max()}")
print(f"   Minimum: {text_len.min()}")
print(f"   Standard deviation: {text_len.std():.2f}")
print()
print("Summary statistics on the number of paragraphs across all blog posts:")
print(f"   Mean: {text_para_len.mean():.2f}")
print(f"   Median: {text_para_len.median():.2f}")
print(f"   Maximum: {text_para_len.max()}")
print(f"   Minimum: {text_para_len.min()}")
print(f"   Standard deviation: {text_para_len.std():.2f}")

In [ ]:
MAX_FEATURES = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_LEN = 1000 # max number of words in a blog post
EMBEDDING_PATH = Path.cwd() / "resources" / "embeddings"
EMBEDDING_FOLDER = EMBEDDING_PATH / "glove.840B.300d"
SPECIFIC_ONLY = False
L2_REG = None

##### Read embeddings

In [ ]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype="float32")

In [ ]:
if (EMBEDDING_FOLDER / "embeddings_index.pkl").is_file():
    with open(EMBEDDING_FOLDER / "embeddings_index.pkl", "rb") as handle:
        embeddings_index = pickle.load(handle)
else:
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FOLDER / "glove.840B.300d.txt"))
    with open(EMBEDDING_FOLDER / "embeddings_index.pkl", "wb") as handle:
        pickle.dump(embeddings_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

##### Preprocess the text data to work well with embeddings

In [ ]:
X_train = pd.Series(X_train_text.copy())
X_test = pd.Series(X_test_text.copy())

In [ ]:
X_train = X_train.apply(lambda x: x.lower())
X_train = X_train.apply(lambda x: clean_apostrophe(x))
X_test = X_train.apply(lambda x: fix_punctuation(x))
X_train = X_train.apply(lambda x: fix_specific(x))

Apply the same transformations to the test set

In [ ]:
X_test = X_test.apply(lambda x: x.lower())
X_test = X_test.apply(lambda x: clean_apostrophe(x))
X_test = X_test.apply(lambda x: fix_punctuation(x))
X_test = X_test.apply(lambda x: fix_specific(x))

##### Tokenize the texts 

In [ ]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(list(X_train))

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train_text)
X_test = tokenizer.texts_to_sequences(X_test_text)

In [ ]:
X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)

##### Process embeddings into a matrix of size `(max_features, embed_size)`

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

In [ ]:
word_index = tokenizer.word_index

if SPECIFIC_ONLY: word_index = {word : i for word, i in word_index.items() if word in specific_vocab}

In [ ]:
nb_words = min(MAX_FEATURES, len(tokenizer.word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= MAX_FEATURES: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

##### Define model

In [ ]:
from keras import regularizers
reg = regularizers.l2(L2_REG) if L2_REG else None

In [ ]:
inp = Input(shape=(MAX_LEN,))
x = Embedding(MAX_FEATURES, embed_size, weights=[embedding_matrix], embeddings_regularizer=reg)(inp)
x = Bidirectional(GRU(64, return_sequences=True, kernel_regularizer=reg))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu", kernel_regularizer=reg)(x)
x = Dropout(0.6)(x)
x = Dense(1, activation="sigmoid")(x)


model = Model(inputs=inp, outputs=x)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

##### Fit model

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=10,
          validation_data=(X_test, y_test), callbacks=[EarlyStopping(min_delta=0.005, patience=2, restore_best_weights=True)])

##### Evaluate model

In [ ]:
train_auc = metrics.roc_auc_score(y_train, model.predict(X_train))
test_auc = metrics.roc_auc_score(y_test, model.predict(X_test))

In [ ]:
results_keras = pd.DataFrame({"method": f"embeddings keras v{SEED}", "train auc": train_auc, "test auc": test_auc},
                             index=[0])

In [ ]:
results_keras

In [ ]:
save_pickle(results_keras, OUTPUT_PATH)

##### Collect results and save for report

In [ ]:
list_keras = read_results(OUTPUT_PATH / "raw", "*keras*.pkl")

In [ ]:
full_results_keras = pd.concat(list_keras)

In [ ]:
full_results_keras

In [ ]:
save_html(df=full_results_keras.drop("base_method", axis=1).sort_values(["test auc"], axis=0, ascending=False),
          name="keras_results_all_appendix",
          out=OUTPUT_PATH,
          index=False)

Aggregate results and save for report

In [ ]:
res = full_results_keras.groupby("base_method").agg({c : ["mean", "std"] for c in ["train auc", "test auc"]})

In [ ]:
out = res.sort_values([("test auc", "mean")], axis=0, ascending=False)
out.index.name = None

In [ ]:
index = pd.MultiIndex.from_product([["train auc", "test auc"], ["mean", "std"]])
out.columns = index

In [ ]:
out

In [ ]:
save_html(df=out,
          name="keras_results_summary",
          out=OUTPUT_PATH,
          index=True)

#### Language model + neural network

In [ ]:
from fastai.datasets import URLs
from fastai.text import TextList
from fastai.basic_data import DatasetType
from fastai.text.learner import text_classifier_learner
from fastai.text.learner import language_model_learner

In [ ]:
PATH = MODEL_PATH / "fastai"
if not PATH.is_dir():
    PATH.mkdir(parents=True)

In [ ]:
MIN_N_TOKENS = 0
BPTT = 1000
MAX_LEN = 2000
SPLIT_TEXTS = False

Optionally, we split the long texts into chunks based on new line characters, train on those and put the results back together in the end

In [ ]:
X_train_full = pd.DataFrame({"text": X_train_text, "label": y_train})
X_test_full = pd.DataFrame({"text": X_test_text, "label": y_test})

if SPLIT_TEXTS:
    X_train_full_exploded = explode_texts(X_train_full)
    X_test_full_exploded = explode_texts(X_test_full)

    X_train_full_exploded["is_test"] = False
    X_test_full_exploded["is_test"] = True

    data_proc = pd.concat([X_train_full_exploded, X_test_full_exploded], axis=0)
else:
    X_train_full["is_test"] = False
    X_test_full["is_test"] = True
    data_proc = pd.concat([X_train_full, X_test_full], axis=0)
    
n_tokens = data_proc["text"].apply(lambda x: len(x.split(" ")))

print("Summary statistics on the number of words across all blog posts:")
print(f"   Mean: {n_tokens.mean():.2f}")
print(f"   Median: {n_tokens.median():.2f}")
print(f"   Maximum: {n_tokens.max()}")
print(f"   Minimum: {n_tokens.min()}")
print(f"   Standard deviation: {n_tokens.std():.2f}")

In [ ]:
data_proc.head()

In [ ]:
data_proc.shape

##### Train and Evaluate the Language Model (Metric : Accuracy)

In [ ]:
data_lm = (TextList.from_df(df=data_proc, path=PATH, cols="text")
             .random_split_by_pct(0.2)
             .label_for_lm()
             .databunch())

In [ ]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, bptt=BPTT, drop_mult=0.5)

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
lr = 5e-2

In [ ]:
learn.fit_one_cycle(2, lr, moms=(0.8, 0.7))

In [ ]:
learn.save("lm_fit_head")

In [ ]:
learn.load("lm_fit_head");

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, lr/10, moms=(0.8, 0.7))

In [ ]:
learn.save("lm_fine_tuned")

In [ ]:
learn.save_encoder("lm_fine_tuned_enc")

##### Train and Evaluate the Classifier (Metric = ROC AUC)

###### Setup

In [ ]:
clf_df = data_proc.copy()[n_tokens >= MIN_N_TOKENS]

In [ ]:
clf_df.shape

In [ ]:
data_clf = (TextList.from_df(df=clf_df, path=PATH, cols=["text"], vocab=data_lm.vocab)
               .split_from_df("is_test")
               .label_from_df(cols="label")
               .databunch(bs=8))

In [ ]:
clf_df.head()

In [ ]:
data_clf.show_batch()

###### Training

In [ ]:
learn = text_classifier_learner(data_clf, bptt=BPTT, max_len=MAX_LEN, drop_mult=0.3)
learn.load_encoder("lm_fine_tuned_enc")
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot();

In [ ]:
lr = 5e-3

In [ ]:
learn.fit_one_cycle(1, lr, moms=(0.8, 0.7))

In [ ]:
learn.save("clf_first_stage")

In [ ]:
learn.load("clf_first_stage");

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(lr/(2.6**4), lr), moms=(0.8, 0.7))

In [ ]:
learn.save("clf_second_stage")

In [ ]:
learn.load("clf_second_stage");

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice((lr/5)/(2.6**4), lr/5), moms=(0.8, 0.7))

In [ ]:
learn.save("clf_third_stage")

In [ ]:
learn.load("clf_third_stage");

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice((lr/10)/(2.6**4), lr/10), moms=(0.8, 0.7))

###### Evaluation

In [ ]:
p1_test = clf_df.loc[clf_df.is_test == True, "text"].apply(lambda t: learn.predict(t)[2].numpy()[1])

In [ ]:
p1_train = clf_df.loc[clf_df.is_test == False, "text"].apply(lambda t: learn.predict(t)[2].numpy()[1])

In [ ]:
#This doesn't seem to produce results consistent with what we get when explicitly calling predict on each sample
#I do need to finish this assignment before looking into this more closely
#Thus, I will use the slower version above for now
#p1_test = learn.get_preds()[0][:, 1].numpy()
#p1_train = learn.get_preds(ds_type=DatasetType.Train)[0][:, 1].numpy()

In [ ]:
if SPLIT_TEXTS:
    print("TRAIN PERFORMANCE")
    evaluate_exploded(clf_df[clf_df.is_test == False], p1_train, y_train)
    print()
    print("TEST PERFORMANCE")
    evaluate_exploded(clf_df[clf_df.is_test == True], p1_test, y_test)
else:
    print("TRAIN PERFORMANCE")
    train_auc = metrics.roc_auc_score(clf_df.loc[clf_df.is_test == False, "label"], p1_train)
    print(f"AUC: {train_auc:.2f}")
    print()
    print("TEST PERFORMANCE")
    test_auc = metrics.roc_auc_score(clf_df.loc[clf_df.is_test == True, "label"], p1_test)
    print(f"AUC: {test_auc:.2f}")
    results_lm_finetune = pd.DataFrame({"method": f"lm fine tuning v{SEED}", "train auc": train_auc, "test auc": test_auc},
                         index=[0])

In [ ]:
results_lm_finetune

In [ ]:
save_pickle(results_lm_finetune, OUTPUT_PATH)

Collect results and save for report

In [ ]:
list_lm_finetune = read_results(OUTPUT_PATH, "*lm*.pkl")

In [ ]:
full_results_lm_finetune = pd.concat(list_lm_finetune)

In [ ]:
full_results_lm_finetune

In [ ]:
save_html(df=full_results_lm_finetune.drop("base_method", axis=1).sort_values(["test auc"], axis=0, ascending=False),
          name="lm_finetune_results_all_appendix",
          out=OUTPUT_PATH,
          index=False)

Summarize results and save summary

In [ ]:
res = full_results_lm_finetune.groupby("base_method").agg({c : ["mean", "std"] for c in ["train auc", "test auc"]})

In [ ]:
out = res.sort_values([("test auc", "mean")], axis=0, ascending=False)
out.index.name = None

In [ ]:
index = pd.MultiIndex.from_product([["train auc", "test auc"], ["mean", "std"]])
out.columns = index

In [ ]:
out

In [ ]:
save_html(df=out,
          name="lm_finetune_results_summary",
          out=OUTPUT_PATH,
          index=True)

#### Google's guide for text classification

The below is based on the [this guide](https://developers.google.com/machine-learning/guides/text-classification/) was attempted after all of the above had been experimented with. Google's guide to building a text classifier suggests a simple heuristic to decide whether to use n-gram based approaches (e.g. those illustrated in 6.1 and 6.2) or sequence based approaches (e.g. using recurrent neural networks as showcased in chapters 6.3 and 6.4):

`if number of samples/number of words per sample < 1500 tokenize as n-grams and use an MLP (or boosted tree, SVM) to classifiy`

`if > 1500 tokenize as sequences and use a sequence model (CNN, RNN) to classify, potentially building on pretrained word embeddings`

As in our case the articles are rather long and we have just a small number of samples, we will follow the first approach.

##### Tokenize texts as unigrams and bigrams with feature selection

In [ ]:
X_train_text_vec, X_test_text_vec = ngram_vectorize(X_train_text, y_train, X_test_text)

##### Build and train an MLP model

In [ ]:
data_in = ((X_train_text, y_train), (X_test_text, y_test))
model = tf_helpers.train_ngram_model(data_in,
                          batch_size=8,
                          dropout_rate=0.3,
                          learning_rate=5e-4,
                          units=32)

In [ ]:
metrics.roc_auc_score(y_train, model.predict(X_train_text_vec))

In [ ]:
metrics.roc_auc_score(y_test, model.predict(X_test_text_vec))

### Final evaluation

#### Run for different random seeds

Run best model for different seeds and save resulting auc, precision, and recall.

In [ ]:
X_train_text, X_test_text = np.array(X_train[TEXT_COL]), np.array(X_test[TEXT_COL])

In [ ]:
best_model = joblib.load(MODEL_PATH / "countvec_full_svc_grid_best.pkl").set_params(svc__random_state=SEED)

In [ ]:
best_model.fit(X_train_text, y_train)

In [ ]:
res_best = get_best_scores(y_test, best_model.predict_proba(X_test_text)[:, 1], f"final_model_svc_countvec_full_v{SEED}")

In [ ]:
res_best

In [ ]:
save_pickle(res_best, OUTPUT_PATH)

Run best benchmark model as a point of comparison.

In [ ]:
X_train_num, X_test_num = np.array(X_train[NUM_COLS]), np.array(X_test[NUM_COLS])

In [ ]:
lr = LogisticRegression(solver="liblinear", random_state=SEED)
lr.fit(X_train_num, y_train)

In [ ]:
res_base = get_best_scores(y_test, lr.predict_proba(X_test_num)[:, 1], f"baseline_model_lr_v{SEED}")

In [ ]:
res_base

In [ ]:
save_pickle(res_base, OUTPUT_PATH)

#### Collect and summarize results

In [ ]:
list_best = read_results(OUTPUT_PATH / "raw", "final_model*.pkl")
full_res_best = pd.concat(list_best)

In [ ]:
full_res_best

In [ ]:
list_base = read_results(OUTPUT_PATH / "raw", "baseline_model*.pkl")
full_res_base = pd.concat(list_base)

In [ ]:
full_res_base

In [ ]:
res = pd.concat([full_res_base, full_res_best])

In [ ]:
res.drop("base_method", axis=1).sort_values("f_0.8", ascending=False)

Save raw results

In [ ]:
save_html(res.drop("base_method", axis=1).sort_values("f_0.8", ascending=False), "final_evaluation_raw_appendix", OUTPUT_PATH)

Aggregate results and save to file

In [ ]:
summary = (res
          .groupby("base_method")
          .agg({c : ["mean", "std"] for c in ["roc_auc", "precision", "recall", "f_0.8"]}))

In [ ]:
summary

In [ ]:
summary.index.name = None
index = pd.MultiIndex.from_product([["roc auc", "precision", "recall", "f_0.8"], ["mean", "std"]])
summary.columns = index

In [ ]:
summary.T

In [ ]:
save_html(summary.T, "final_evaluation_summary", OUTPUT_PATH)